In [6]:
import os
import git
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm_notebook as tqdm
import ast
import re
import pickle
import matplotlib.pyplot as plt
import git
import pickle

In [3]:
# get the git repo
def get_git_root(path):
	git_repo = git.Repo(path, search_parent_directories=True)
	git_root = git_repo.git.rev_parse("--show-toplevel")
	return git_root

In [18]:
model_dir = os.path.join(get_git_root(os.getcwd()),"models")
input_dir = os.path.join(get_git_root(os.getcwd()),"input")

In [19]:
poly_before = pickle.load(open(os.path.join(model_dir,"poly_users","poly_before.pkl"),"rb"))

In [20]:
len(poly_before)

238

In [26]:
## getting their first juul tweets
juul_first = pd.read_csv(os.path.join(input_dir,"user_first_juul.csv"),index_col=0)
juul_first = juul_first.loc[juul_first.userID.isin(poly_before)]

In [37]:
## getting their first weed tweet
timeline = pd.read_csv(os.path.join(input_dir,"labelled_data","tweets_predicted.csv"),index_col=0,lineterminator="\n")
timeline = timeline.loc[timeline.userID.isin(poly_before)]

In [48]:
## getting weed pattern
weed_words = pickle.load(open(os.path.join(model_dir,"weed_words.pkl"),"rb"))

In [49]:
weed_words = [(" " + word + " ") for word in weed_words]
weed_pattern = "|".join(weed_words)

In [52]:
## weed_pattern
weed_tweets = timeline.loc[timeline.tweetText.str.contains(weed_pattern)]

In [64]:
first_weed = list()
for user in poly_before:
    user_tweets = weed_tweets.loc[weed_tweets.userID == user]
    user_tweets = user_tweets.sort_values(by='tweetCreatedAt',ascending=True)
    tweet_first = list(user_tweets["tweetCreatedAt"].head(1))
    if tweet_first:
        first_weed.append(tweet_first[0])

## Dump the weed Pattern in . model dir

In [70]:
filename = os.path.join(model_dir,"weed_pattern.pkl")
with open(filename,"wb") as f:
    pickle.dump((filename),f)